In [1]:
import sys
sys.path.append("helios-server/")
import helios.models as models
import helios.views as views
from helios_auth import models as auth_models
from helios.models import Election, CastVote, Voter, Trustee
from helios.workflows.homomorphic import EncryptedAnswer, EncryptedVote, Tally
import json
import uuid

In [2]:
user = auth_models.User.objects.get(user_id='ben@adida.net', user_type='google')

In [3]:
election, _ = models.Election.get_or_create( # doesn't recreate it if these specs already exist in some election, I think
                                short_name='my_test',
                                name='My Test',
                                description='Please let this work....',
                                admin=user,
                                # questions=questions,
                                )
print(election.created_at)

2022-10-27 11:27:28.679897


In [4]:
print(election.num_voters)

1


In [5]:
# voter = models.Voter.register_user_in_election(user, election) # if you do this multiple times the user will be a voter multiple times...
print(election.num_voters)

1


In [6]:
print(election.num_trustees)

1


In [88]:
# election.generate_trustee(views.ELGAMAL_PARAMS) # only needs to be done once per Election creation
print(election.num_trustees)

1


In [8]:
dir(election)

['DoesNotExist',
 'ELECTION_TYPES',
 'Meta',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check_unique_together',
 '_do_insert',
 '_do_update',
 

In [9]:
election.registration_status_pretty

'Closed'

In [11]:
questions = [{"answer_urls":[None, None, None],
              "answers": ["A", "B", "C"],
              "choice_type": "approval",
              "max": 1,
              "min": 1,
              "question": "A, B, or C?",
              "result_type": "absolute",
              "short_name": "W?",
              "tally_type": "homomorphic",
             }]

election.save_questions_safely(questions)

True

In [7]:
election.questions

[{'answer_urls': [None, None, None],
  'answers': ['A', 'B', 'C'],
  'choice_type': 'approval',
  'max': 1,
  'min': 1,
  'question': 'A, B, or C?',
  'result_type': 'absolute',
  'short_name': 'W?',
  'tally_type': 'homomorphic'}]

In [9]:
print(election.frozen_at)

2022-10-28 15:11:35.216126


In [12]:
election.pretty_result

[{'question': 'W?',
  'answers': [{'answer': 'A', 'count': 0, 'winner': False},
   {'answer': 'B', 'count': 0, 'winner': False},
   {'answer': 'C', 'count': 1, 'winner': False}]}]

In [21]:
election.num_cast_votes

0

In [84]:
# voter = Voter(uuid=str(uuid.uuid1()), election=election, user=user)
# voter.save()

In [12]:
election.num_voters

1

In [82]:
for v in Voter.get_by_election(election):
    v.delete()

In [37]:
election.num_voters

1

In [173]:
answer = EncryptedAnswer.fromElectionAndAnswer(election, 0, [2])

In [174]:
answer.answer

[2]

In [175]:
answer.choices

In [176]:
answer.answer

[2]

In [177]:
# gotta figure out why there are two possible plaintexts (shouldn't there be 1?)
answer.verify(election.public_key)

bad number of proofs (expected 2, found 1)


False

In [178]:
answer.generate_plaintexts(election.public_key)

In [179]:
answer.individual_proofs[0]

In [16]:
vote = EncryptedVote.fromElectionAndAnswers(election, [[0]]) # first question, answer is at index 2

In [17]:
vote.verify(election) # nice!

True

In [17]:
vote

In [205]:
# voter.vote = vote # I think wrong

In [19]:
vote.answers[0].answer

[0]

In [20]:
voter = Voter.get_by_user(user)[0]

In [21]:
vote.hash

'rp6SeN3RJq4owkURCvSoI9Wkutb/s8La66ES7NObcuY'

In [22]:
castvote = CastVote(vote=vote, vote_hash=vote.hash, voter=voter)

In [23]:
voter.store_vote(castvote)

In [24]:
election.compute_tally()

In [43]:
election.encrypted_tally.decrypt_and_prove()

TypeError: decrypt_and_prove() missing 1 required positional argument: 'sk'

In [84]:
election.trustee_set

<django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager at 0x10a1ff310>

In [26]:
election.ready_for_decryption_combination()

True

In [27]:
election.helios_trustee_decrypt()

In [28]:
election.combine_decryptions()

In [29]:
election.result

[[1, 0, 0]]

In [40]:
election.get_helios_trustee().verify_decryption_proofs()

True

In [46]:
voter.store_vote(vote)

AttributeError: 'EncryptedVote' object has no attribute 'vote'

In [22]:
tally = Tally(election=election)
tally.add_vote(vote)

In [23]:
tally

In [ ]:
# pretty good so far!!

In [47]:
election.num_cast_votes

0

In [28]:
election.ready_for_decryption()

False

In [235]:
tally.decrypt_and_prove()

TypeError: decrypt_and_prove() missing 1 required positional argument: 'sk'

In [212]:
cv.issues(election)

AttributeError: 'str' object has no attribute 'election_uuid'

In [107]:
cv.objects.filter(voter = voter)

AttributeError: Manager isn't accessible via CastVote instances

In [79]:
pulled_logs = [l.log for l in election.get_log().all()]
print(pulled_logs)

['frozen', 'Trustee Ben for Helios added']


In [18]:
import sys
sys.path.append("helios-server/")
import helios
# import helios.models as models
sys.path.append("helios-server/helios/crypto/")
from numtheory import *
from utils import *
from algs import *
sys.path.append("helios-server/helios/workflows/")
from homomorphic import *

import json

In [8]:
election = json.load(open("helios-server/helios/fixtures/election.json", "r"))[0]

NameError: name 'json' is not defined

In [7]:
election

<Election: Election object (1)>

In [ ]:
self.election = models.Election.objects.get(short_name='test')

In [26]:
tally = Tally()
tally.init_election(election)

AttributeError: 'dict' object has no attribute 'questions'

In [22]:
tally

In [14]:
x = EncryptedAnswer(choices=["A","B"], answer="A")

In [16]:
x.choices

['A', 'B']

In [17]:
x.answer

'A'